In [ ]:
import cv2
import numpy as np
import os



import keras
from keras.applications import DenseNet121
from keras.models import Model, Sequential
from keras import layers
from keras.callbacks import EarlyStopping
from keras.metrics import AUC
# from keras.metrics import f1_score
from keras.metrics import Precision
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import F1Score, AUC, Precision, Recall
from tensorflow.keras.layers import MaxPooling2D, Dropout



from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer




from PIL import Image

In [ ]:
import os
import librosa
import numpy as np
from skimage.transform import resize

def get_mel_spectrogram(file_directory, sr=22050, desired_shape=(244, 244, 3)):
    """
    Get mel-spectrogram (db) with desired shape
    Input: file_directory: str, sr: int, desired_shape: tuple
    Output: mel_spec: numpy array
    """
    src, sc = librosa.load(file_directory, sr=22050)
    mel_gram = librosa.feature.melspectrogram(y=src, sr=22050)

    # Convert to decibels
    mel_spec_db = librosa.power_to_db(mel_gram, ref=np.max)

    # Resize to desired shape
    mel_spec_resized = resize(mel_spec_db, desired_shape)

    return mel_spec_resized

def process_audio_files(audio_dir):
    ''' Process audio files in the specified directory and return log-mel spectrograms and labels.

    Parameters:
    ----------
    audio_dir: str
        Path to the directory containing audio files.

    Returns:
    -------
    tuple
        A tuple containing a list of log-mel spectrograms and corresponding labels.
    '''

    # List to store log-mel spectrograms and corresponding labels
    data = []
    labels = []

    # Iterate over the subdirectories (genres)
    for genre_folder in os.listdir(audio_dir):
        genre_path = os.path.join(audio_dir, genre_folder)
        if os.path.isdir(genre_path):  # Check if it's a directory
            # Iterate over all audio files in the genre directory
            for filename in os.listdir(genre_path):
                if filename.endswith('.wav'):  # Assuming all audio files are in .wav format
                    file_path = os.path.join(genre_path, filename)

                    # Compute log-mel spectrogram for the current audio file
                    log_mel_spectrogram = get_mel_spectrogram(file_path)

                    # Append log-mel spectrogram to data list
                    data.append(log_mel_spectrogram)

                    # Append label to labels list (genre folder name)
                    labels.append(genre_folder)

    return data, labels

# Directory containing audio files
audio_dir = '/content/drive/MyDrive/Music_Project/music_data/VNTM3'

# Process audio files and retrieve log-mel spectrograms and labels
data, labels = process_audio_files(audio_dir)


In [ ]:
# Split data into training and combined validation-evaluation set
x_train, x_combined, y_train, y_combined = train_test_split(data, labels, test_size=0.2, random_state=42, shuffle=True, stratify =labels)

# Split combined set into validation and evaluation sets
x_validate, x_test, y_validate, y_test = train_test_split(x_combined, y_combined, test_size=0.5, random_state=42, shuffle=True, stratify =y_combined)


In [ ]:
label_encoder = LabelEncoder()
label_binarizer = LabelBinarizer()
# Fit and transform training labels
y_train_one_hot = label_binarizer.fit_transform(y_train)
y_validate_one_hot = label_binarizer.transform(y_validate)
y_test_one_hot = label_binarizer.transform(y_test)
y_test_encoded = label_encoder.fit_transform(y_test)
# Fit and transform evaluation labels
y_test_encoded = label_encoder.fit_transform(y_test)
y_test_one_hot = label_binarizer.transform(y_test)

In [ ]:
X_test = np.array(x_test, dtype=np.float32)
X_train = np.array(x_train, dtype = np.float32)
X_validate = np.array(x_validate, dtype=np.float32)

In [ ]:
base_model = DenseNet121(weights='imagenet',
                         include_top=False,
                         input_shape=(244,244,3))

early = EarlyStopping(monitor='val_loss',
                      min_delta=0.0001,
                      mode='min',
                      verbose=0,
                      patience=5,
                      baseline= None,
                      restore_best_weights= True)

top_model = Sequential([
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    Dropout(0.2),
    layers.Dense(5, activation='softmax')
])


model = Model(inputs=base_model.input,
              outputs=top_model(base_model.output))

# Freeze base layers to prevent retraining
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=["accuracy"])
#model.summary()

model.fit(X_train, y_train_one_hot,
          epochs=100,
          validation_data=(X_validate, y_validate_one_hot),
          callbacks=[early],
          shuffle=True,
          verbose=1,
          batch_size=32)

29084464/29084464 [==============================] - 1s 0us/step
Epoch 1/100
63/63 [==============================] - 20s 147ms/step - loss: 1.1805 - accuracy: 0.6218 - val_loss: 0.5279 - val_accuracy: 0.8320
Epoch 2/100
63/63 [==============================] - 3s 51ms/step - loss: 0.6179 - accuracy: 0.7774 - val_loss: 0.4778 - val_accuracy: 0.7960
Epoch 3/100
63/63 [==============================] - 3s 51ms/step - loss: 0.4912 - accuracy: 0.8229 - val_loss: 0.3345 - val_accuracy: 0.8920
Epoch 4/100
63/63 [==============================] - 3s 51ms/step - loss: 0.4269 - accuracy: 0.8449 - val_loss: 0.2771 - val_accuracy: 0.9120
Epoch 5/100
63/63 [==============================] - 3s 48ms/step - loss: 0.3988 - accuracy: 0.8549 - val_loss: 0.4370 - val_accuracy: 0.8400
Epoch 6/100
63/63 [==============================] - 3s 49ms/step - loss: 0.3715 - accuracy: 0.8679 - val_loss: 0.3225 - val_accuracy: 0.8880
Epoch 7/100
63/63 [==============================] - 3s 52ms/step - loss: 0.3384 

In [ ]:
predict = model.predict(X_test)


# Convert one-hot encoded predicted labels back to original labels
test_predictions_labels = label_encoder.inverse_transform(np.argmax(predict, axis=1))
# Convert one-hot encoded predicted labels back to original labels
test_predictions_labels_encoded = label_encoder.inverse_transform(np.argmax(predict, axis=1))


# Calculate AUC_ROC
auc_roc = AUC()
auc_roc.update_state(y_test_one_hot, predict)
print("AUC ROC:", auc_roc.result().numpy())

# Calculate F1 Score
f1 = F1Score()
f1.update_state(y_test_one_hot, predict)
print("F1 Score:", np.mean(f1.result().numpy()))

# Calculate Precision
pre = Precision()
pre.update_state(y_test_one_hot, predict)
print("Precision Score", pre.result().numpy())

# Accuracy Score
acc = accuracy_score(y_test, test_predictions_labels)
print("Accuracy Score", acc)

print("Recall Score:", recall_score(y_test_encoded, np.argmax(predict, axis=1), average='macro'))

8/8 [==============================] - 2s 38ms/step
AUC ROC: 0.995938
F1 Score: 0.95263547
Precision Score 0.9508197
Accuracy Score 0.952
Recall Score: 0.952


In [ ]:
test_predictions_labels_encoded_list = test_predictions_labels_encoded.tolist()

In [ ]:
# Print results
print("Sample\t\tTrue Label\tPredicted Label")
print("-" * 40)
for i in range(len(y_test)):
    print(f"{i+1}\t\t{y_test[i]}\t\t{test_predictions_labels_encoded_list[i]}")


Sample		True Label	Predicted Label
----------------------------------------
1		cheo		cheo
2		cailuong		cailuong
3		cheo		cheo
4		chauvan		chauvan
5		catru		catru
6		hatxam		hatxam
7		cheo		cheo
8		cailuong		cailuong
9		hatxam		hatxam
10		cailuong		cailuong
11		cailuong		cailuong
12		hatxam		hatxam
13		chauvan		chauvan
14		cheo		cheo
15		cheo		cheo
16		catru		catru
17		cheo		cheo
18		hatxam		hatxam
19		hatxam		chauvan
20		chauvan		chauvan
21		catru		catru
22		chauvan		chauvan
23		hatxam		hatxam
24		hatxam		hatxam
25		hatxam		hatxam
26		chauvan		chauvan
27		catru		catru
28		chauvan		chauvan
29		cheo		hatxam
30		cheo		cheo
31		hatxam		hatxam
32		chauvan		chauvan
33		hatxam		hatxam
34		hatxam		chauvan
35		cailuong		cailuong
36		cailuong		cailuong
37		catru		catru
38		chauvan		hatxam
39		cheo		cheo
40		chauvan		chauvan
41		catru		catru
42		hatxam		hatxam
43		cheo		cheo
44		cheo		cheo
45		chauvan		chauvan
46		hatxam		hatxam
47		hatxam		hatxam
48		hatxam		hatxam
49		cheo		cheo
50		cailuong		c

In [ ]:
# # Define the checkpoint filepath
# checkpoint_dir = '/content/drive/MyDrive/Music_Project/music_data/checkpoint/DenseNet121'
# os.makedirs(checkpoint_dir, exist_ok=True)

# # Create a ModelCheckpoint callback
# checkpoint_filepath = os.path.join(checkpoint_dir, 'Densenet')
# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     save_weights_only=True,  # Save only the model weights
#     monitor='val_loss',  # Monitor validation loss
#     mode='min',  # Save the model when validation loss is minimized
#     save_best_only=True  # Save only the best model
# )

# # Assuming you have already trained and evaluated your model
# # model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[checkpoint_callback])

# # After training, save the entire model
# model.save('/content/drive/MyDrive/Music_Project/music_data/checkpoint/DenseNet121/densenet121_model_10s.h5')

# # Also, save only the weights of the model
# model.save_weights('/content/drive/MyDrive/Music_Project/music_data/checkpoint/DenseNet121/densenet121_weights_10s.h5')


In [ ]:

# # Tải checkpoint đã lưu
# checkpoint_path = "/content/drive/MyDrive/Music_Project/music_data/checkpoint/DenseNet121/densenet121_weights.h5"
# model_path = r"/content/drive/MyDrive/Music_Project/music_data/checkpoint/DenseNet121/densenet121_model.h5"
# model = tf.keras.models.load_model(model_path)
# model.load_weights(checkpoint_path)

# # Đánh giá mô hình sau khi tải lại checkpoint

# predict = model.predict(x_test)


# # Convert one-hot encoded predicted labels back to original labels
# test_predictions_labels = label_encoder.inverse_transform(np.argmax(predict, axis=1))
# # Convert one-hot encoded predicted labels back to original labels
# test_predictions_labels_encoded = label_encoder.inverse_transform(np.argmax(predict, axis=1))


# # Calculate AUC_ROC
# auc_roc = AUC()
# auc_roc.update_state(y_test_one_hot, predict)
# print("AUC ROC:", auc_roc.result().numpy())

# # Calculate F1 Score
# f1 = F1Score()
# f1.update_state(y_test_one_hot, predict)
# print("F1 Score:", np.mean(f1.result().numpy()))

# # Calculate Precision
# pre = Precision()
# pre.update_state(y_test_one_hot, predict)
# print("Precision Score", pre.result().numpy())

# # Accuracy Score
# acc = accuracy_score(y_test, test_predictions_labels)
# print("Accuracy Score", acc)

# print("Recall Score:", recall_score(y_test_encoded, np.argmax(predict, axis=1), average='macro'))